### NLP Model for NCRP 


Basic data exploration

In [41]:
import pandas as pd

df = pd.read_csv('train.csv')

df.head()

,category,sub_category,crimeaditionalinfo
0,Online and Social Media Related Crime,Cyber Bullying Stalking Sexting,I had continue received random calls and abusi...
1,Online Financial Fraud,Fraud CallVishing,The above fraudster is continuously messaging ...
2,Online Gambling Betting,Online Gambling Betting,He is acting like a police and demanding for m...
3,Online and Social Media Related Crime,Online Job Fraud,In apna Job I have applied for job interview f...
4,Online Financial Fraud,Fraud CallVishing,I received a call from lady stating that she w...


In [42]:
len(df), len(df['category'].unique()), len(df['sub_category'].unique())

(93686, 15, 36)

In [43]:
empty_cells_count = df['category'].replace(r"^\s*$", pd.NA, regex=True).isnull().sum()
print(f"Total number of empty cells in 'category': {empty_cells_count}")

Total number of empty cells in 'category': 0


In [44]:
empty_cells_count = df['sub_category'].replace(r"^\s*$", pd.NA, regex=True).isnull().sum()
print(f"Total number of empty cells in 'sub_category': {empty_cells_count}")

Total number of empty cells in 'sub_category': 6591


In [45]:
df['category'].value_counts()

category
Online Financial Fraud                                  57434
Online and Social Media Related Crime                   12140
Any Other Cyber Crime                                   10878
Cyber Attack/ Dependent Crimes                           3608
RapeGang Rape RGRSexually Abusive Content                2822
Sexually Obscene material                                1838
Hacking  Damage to computercomputer system etc           1710
Sexually Explicit Act                                    1552
Cryptocurrency Crime                                      480
Online Gambling  Betting                                  444
Child Pornography CPChild Sexual Abuse Material CSAM      379
Online Cyber Trafficking                                  183
Cyber Terrorism                                           161
Ransomware                                                 56
Report Unlawful Content                                     1
Name: count, dtype: int64

In [46]:
df['sub_category'].value_counts()

sub_category
UPI Related Frauds                                                      26856
Other                                                                   10878
DebitCredit Card FraudSim Swap Fraud                                    10805
Internet Banking Related Fraud                                           8872
Fraud CallVishing                                                        5803
Cyber Bullying  Stalking  Sexting                                        4089
EWallet Related Fraud                                                    4047
FakeImpersonating Profile                                                2299
Profile Hacking Identity Theft                                           2073
Cheating by Impersonation                                                1988
Unauthorised AccessData Breach                                           1114
Online Job Fraud                                                          912
DematDepository Fraud                              

In [47]:
category_dict = df.groupby('category')['sub_category'].unique().apply(list).to_dict()
category_dict

{'Any Other Cyber Crime': ['Other'],
 'Child Pornography CPChild Sexual Abuse Material CSAM': [nan],
 'Cryptocurrency Crime': ['Cryptocurrency Fraud'],
 'Cyber Attack/ Dependent Crimes': ['Data Breach/Theft',
  'Denial of Service (DoS)/Distributed Denial of Service (DDOS) attacks',
  'Malware Attack',
  'Hacking/Defacement',
  'SQL Injection',
  'Ransomware Attack',
  'Tampering with computer source documents'],
 'Cyber Terrorism': ['Cyber Terrorism'],
 'Hacking  Damage to computercomputer system etc': ['Email Hacking',
  'Unauthorised AccessData Breach',
  'Website DefacementHacking',
  'Damage to computer computer systems etc',
  'Tampering with computer source documents'],
 'Online Cyber Trafficking': ['Online Trafficking'],
 'Online Financial Fraud': ['Fraud CallVishing',
  'UPI Related Frauds',
  'Internet Banking Related Fraud',
  'DebitCredit Card FraudSim Swap Fraud',
  'EWallet Related Fraud',
  'Business Email CompromiseEmail Takeover',
  'DematDepository Fraud'],
 'Online Ga

In [51]:
df['category'] = df['category'].fillna('nan')
df['sub_category'] = df['sub_category'].fillna('nan')
df['crimeaditionalinfo'] = df['crimeaditionalinfo'].fillna('nan')

In [52]:
df.isnull().sum()

category              0
sub_category          0
crimeaditionalinfo    0
dtype: int64